# 경사 하강법을 이용한 얕은 신경망 학습


In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [2]:
epochs = 1000


## 네트워크 구조 정의
### 얕은 신경망
#### 입력 계층 : 2, 은닉 계층 : 128 (Sigmoid activation), 출력 계층 : 10 (Softmax activation)

In [34]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128, input_dim=2, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self,x, trianing = None, mask = None):
        x = self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [47]:
# 모든 tensor연산을 파이썬 문법으로 최적화
@tf.function

def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss  = loss_object(labels, predictions)
    
    # 스칼라를 모델에 있는 값으로 미분을 해주는 것
    # df(x) / dx
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_metric(labels, predictions) # 정답과 예측값을 비교


    

## 데이터셋 생성, 전처리

In [48]:
# 10개의 클래스점 생성
np.random.seed(0)

pts = list()
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10,2))

# 랜덤한 100개의 값들을 뿌르기
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape))
        labels.append(label)

pts = np.stack(pts, axis=0).astype(np.float32)
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(10000).batch(16)

## 모델 생성

In [49]:
model = MyModel()

## 손실 함수 및 최적화 알고리즘 설정
### CrossEntropy, Adam Optimizer

In [50]:
# 몇번째가 1인지를 확인
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정
### Accuracy

In [51]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프

In [53]:
for epoch in range(epochs):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
    template = 'Epoch :{}, Loss : {}, Accuracy :{}'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result() * 100))

:703, Loss : 0.2728997766971588, Accuracy :89.38497924804688
Epoch :704, Loss : 0.27286872267723083, Accuracy :89.38566589355469
Epoch :705, Loss : 0.2728331983089447, Accuracy :89.38713836669922
Epoch :706, Loss : 0.27280497550964355, Accuracy :89.38826751708984
Epoch :707, Loss : 0.2727730870246887, Accuracy :89.3896255493164
Epoch :708, Loss : 0.2727401554584503, Accuracy :89.3905258178711
Epoch :709, Loss : 0.2727051377296448, Accuracy :89.39143371582031
Epoch :710, Loss : 0.2726723253726959, Accuracy :89.3925552368164
Epoch :711, Loss : 0.2726403772830963, Accuracy :89.3934555053711
Epoch :712, Loss : 0.27260905504226685, Accuracy :89.39446258544922
Epoch :713, Loss : 0.2725745439529419, Accuracy :89.39579772949219
Epoch :714, Loss : 0.27254718542099, Accuracy :89.39635467529297
Epoch :715, Loss : 0.2725166976451874, Accuracy :89.39724731445312
Epoch :716, Loss : 0.2724872827529907, Accuracy :89.39791107177734
Epoch :717, Loss : 0.2724570035934448, Accuracy :89.39846801757812
Epoc

## 데이터셋 및 학습 파라미터 저장

In [54]:
np.savez_compressed('ch2_dataset.npz', inputs = pts, labels=labels)

W_h, b_h = model.d1.get_weights() 
W_o, b_o = model.d2.get_weights()

W_h = np.transpose(W_h)
W_o = np.transpose(W_o)

np.savez_compressed('ch2_parameters.npz', 
                    W_h = W_h,
                    b_h = b_h,
                    W_o = W_o,
                    b_o = b_o)

### 저장된 데이터를 통해 part2에서한 시각화 가능